# Data Mining / Prospecção de Dados

## Sara C. Madeira, 2024/2025

# Project 2 - Classification in Temporal Data using Sequential Pattern Mining

## Logistics 
**_Read Carefully_**

**Students should work in teams of 3 people**. 

Groups with less than 3 people might be allowed (with valid justification), but will not have better grades for this reason. 

The quality of the project will dictate its grade, not the number of people working.

**The project's solution should be uploaded in Moodle before the end of `June, 8th (23:59)`.** 

Students should **upload a `.zip` file** containing a folder with all the files necessary for project evaluation. 
Groups should be registered in [Moodle](https://moodle.ciencias.ulisboa.pt/mod/groupselect/view.php?id=139096) and the `zip` file should be identified as `PDnn.zip` where `nn` is the number of your group.

**It is mandatory to produce a Jupyter notebook containing code and text/images/tables/etc describing the solution and the results. Projects not delivered in this format will not be graded. You can use `PD_202425_P2.ipynb` as template. In your `.zip` folder you should also include an HTML version of your notebook with all the outputs.**

**Decisions should be justified and results should be critically discussed.** 

Remember that **your notebook should be as clear and organized as possible**, that is, **only the relevant code and experiments should be presented, not everything you tried and did not work, or is not relevant** (that can be discussed in the text, if relevant)! Tables and figures can be used together with text to summarize results and conclusions, improving understanding, readability and concision. **More does not mean better! The target is quality not quantity!**

_**Project solutions containing only code and outputs without discussions will achieve a maximum grade of 10 out of 20.**_

## Dataset and Tools

Amyotrophic Lateral Sclerosis (ALS) is a devastating neurodegenerative disease causing rapid degeneration of motor neurons and usually leading to death by respiratory failure. Since there is no cure, treatment’s goal is to improve symptoms and prolong survival. Non-invasive Ventilation (NIV) has been shown to extend life expectancy and improve quality of life, thus it is key to effectively predict if ALS patients will be eligible for NIV in the near future based on disease progression. In this context, Martins et al. (2021) proposed to learn prognostic models using disesase progression patterns (https://ieeexplore.ieee.org/document/9426397), and formulated the following prognostic prediction problem (schematized in Figure below): given a specific ALS patient static data collected at diagnosis and temporal data from disease follow-up, can we effectively use these clinical evaluations to predict if this patient will require NIV within k days of last evaluation?

<img src="prognostic_problem.png" alt="Prognostic Prediction" style="width: 500px;"/>

In this project, we will perform a reduced part of the work published by Martins et al. to learn a machine learning model (classifier) able to predict the need for NIV in a time window of 180 days given static and temporal data and sequential patterns as features.

The dataset to be analysed was obtained from the Lisbon ALS database, containing clinical data from ALS patients collected during their follow-up at the hospital. To reduce the preprocessing steps relatively to what was done in the paper and should be done in the real scenario the following **preprocessed datasets** are already provided:

1. `Dataset_Static_Features.csv` - each row is a patient with a REF id described by a set of features collected at diagnosis time. These features are called static since their are not collected over time. Duplicated patients should be deleted.

<img src="static_example.png" alt="Temporal Data" style="width: 1000px;"/>

2. `Dataset_Temporal_Features.csv` - each patient has a set of rows (snapshots), each corresponding to a visit at the hospital and the values collected for a set of temporal features collected over time. The rows per patient REF are sorted in chonologic order, such that the first row and the last row of a REF correspond, respectively to the data collected in the first and last visit at the hospital. In the example below, the patient REF=9 has temporal data describing 5 clinical evaluations (time-points).

<img src="temporal_example.png" alt="Temporal Data" style="width: 500px;"/>

3. `Dataset_NIV_Evolution_180.cvs`- each patient has a set of rows with the true value of NIV (yes or no) 180 days in the future. These are the class labels to be later used to train the classifier. Patient REF=9 does not evolve to need NIV 180 after time-points 1 to 4 but evolves to NIV 180 after time-point 5.

<img src="NIV_evolution_example.png" alt="Temporal Data" style="width: 100px;"/>

In this context, the project has **2 main tasks**:
1. Learn disease progression patterns from the temporal data using temporal pattern mining
2. Learn a classifier to predict NIV using temporal patterns as features together with the static features


**In this project you should use [Python 3](https://www.python.org), [Jupyter Notebook](http://jupyter.org), [`SPMF`](https://www.philippe-fournier-viger.com/software.php) for temporal pattern mining, and [`Scikit-learn`](https://scikit-learn.org/stable/) for classification.**


## Team Identification

**GROUP NN**

Students:

* Student 1 - n_student1
* Student 2 - n_student2
* Student 3 - n_student3

## 1. Learn disease progression patterns from the temporal data using temporal pattern mining

In this first task you should load and preprocessed the dataset **`Dataset_Temporal_Features.csv`** in order to compute sequential patterns for each patient. 

For that, you first need to load and preprocess this dataset and then transform the temporal data into a **sequence database**.

You should consider **a minimum of 2 time-points** and a **maximum of 5 time-points** per patient.

The sequential pattern mining algorithm `Fourier08` (https://www.philippe-fournier-viger.com/spmf/ClosedSequentialPatterns_TimeConstraints.php), an extension of `PrefixSpan` able to **deal with time** and compute **close patterns**, should be used.

### 1.1. Load and Preprocess Dataset

- Remove all patients (REFs) with missing values in the temporal features.
- Remove all patients with less than 2 time-points.
- For the patients with more than 5 time-points, keep only the first 5.
- In order to reduce the number of patterns to be generated agregate the values of the temporal features using the following intervals (4 intervals for each feature as in Martins et al): 
  1) Intervals for `ALSFRSb, ALSFRSsUL, ALSFRSsT, ALSFRSsLL, and R: <4 [4,8[ [8,12[ 12`;
  2) Intervals for `ALSFRS-R: <12  [12,24[  [24,36[  >=36`

In [1]:
# Write code in cells like this
# ....
import pandas as pd
import os
from spmf import Spmf

In [2]:
temporal_df = pd.read_csv('Dataset_Temporal_Features.csv', delimiter=';')

In [3]:
temporal_df.head(3)

,REF,ALSFRSb,ALSFRSsUL,ALSFRSsT,ALSFRSsLL,R,ALSFRS-R
0,2,12.0,8.0,6.0,4.0,12.0,42.0
1,2,12.0,8.0,6.0,4.0,12.0,42.0
2,2,12.0,8.0,6.0,3.0,12.0,41.0


In [4]:
#removing rows with NaN values
print(f"Number of nulls per column\n{temporal_df.isnull().sum()}")
temporal_df = temporal_df.dropna()
print(f"\nNumber of nulls per column\n{temporal_df.isnull().sum()}")

Number of nulls per column
REF            0
ALSFRSb      318
ALSFRSsUL    329
ALSFRSsT     329
ALSFRSsLL    329
R            366
ALSFRS-R     381
dtype: int64

Number of nulls per column
REF          0
ALSFRSb      0
ALSFRSsUL    0
ALSFRSsT     0
ALSFRSsLL    0
R            0
ALSFRS-R     0
dtype: int64


In [5]:
ref_counts = temporal_df['REF'].value_counts()
print(f"\nReference counts:\n{ref_counts}")


Reference counts:
REF
723     26
269     25
608     24
395     24
981     23
        ..
1162     1
1453     1
110      1
519      1
521      1
Name: count, Length: 957, dtype: int64


In [6]:
#print the unique values in all columns except 'REF'
for col in temporal_df.columns:
    if col != 'REF':
        unique_values = temporal_df[col].unique()
        print(f"\nUnique values in column '{col}':\n{unique_values}")


Unique values in column 'ALSFRSb':
[12.  3. 10.  9.  4. 11.  7.  8.  6.  5.  2.  0.  1.]

Unique values in column 'ALSFRSsUL':
[8. 5. 6. 3. 4. 2. 7. 1. 0.]

Unique values in column 'ALSFRSsT':
[6. 4. 7. 2. 8. 3. 5. 1. 0.]

Unique values in column 'ALSFRSsLL':
[4. 3. 6. 8. 5. 7. 1. 0. 2.]

Unique values in column 'R':
[12. 11. 10.  7.  9.  8.]

Unique values in column 'ALSFRS-R':
[42. 41. 36. 37. 38. 32. 46. 45. 44. 31. 27. 40. 39. 35. 30. 33. 47. 43.
 29. 28. 26. 34. 25. 22. 24. 23. 17. 20. 19. 21. 48. 16. 18. 15. 14. 12.
 11. 13.]


In [7]:
temporal_df = temporal_df.groupby('REF').filter(lambda x: len(x) >= 2)

In [8]:
temporal_df = temporal_df.groupby('REF').head(5)

In [9]:
def alsfrs_intervals(val):
    if val < 4:
        return '<4'
    elif 4 <= val < 8:
        return '[4,8['
    elif 8 <= val < 12:
        return '[8,12['
    else:
        return '12'

In [10]:
cols = ['ALSFRSb', 'ALSFRSsUL', 'ALSFRSsT', 'ALSFRSsLL', 'R']
for col in cols:
    temporal_df[col] = temporal_df[col].apply(alsfrs_intervals)

In [11]:
temporal_df.head(3)

,REF,ALSFRSb,ALSFRSsUL,ALSFRSsT,ALSFRSsLL,R,ALSFRS-R
0,2,12,"[8,12[","[4,8[","[4,8[",12,42.0
1,2,12,"[8,12[","[4,8[","[4,8[",12,42.0
2,2,12,"[8,12[","[4,8[",<4,12,41.0


In [12]:
def alsfrs_r(val):
    if val < 12:
        return '<12'
    elif 12 <= val < 24:
        return '[12,24['
    elif 24 <= val < 36:
        return '[24,36['
    else:
        return '>=36'

In [13]:
temporal_df['ALSFRS-R'] = temporal_df['ALSFRS-R'].apply(alsfrs_r)

In [14]:
temporal_df.head(3)

,REF,ALSFRSb,ALSFRSsUL,ALSFRSsT,ALSFRSsLL,R,ALSFRS-R
0,2,12,"[8,12[","[4,8[","[4,8[",12,>=36
1,2,12,"[8,12[","[4,8[","[4,8[",12,>=36
2,2,12,"[8,12[","[4,8[",<4,12,>=36


In [15]:
#print the unique values in all columns except 'REF'
for col in temporal_df.columns:
    if col != 'REF':
        unique_values = temporal_df[col].unique()
        print(f"\nUnique values in column '{col}':\n{unique_values}")


Unique values in column 'ALSFRSb':
['12' '[8,12[' '[4,8[' '<4']

Unique values in column 'ALSFRSsUL':
['[8,12[' '[4,8[' '<4']

Unique values in column 'ALSFRSsT':
['[4,8[' '<4' '[8,12[']

Unique values in column 'ALSFRSsLL':
['[4,8[' '<4' '[8,12[']

Unique values in column 'R':
['12' '[8,12[']

Unique values in column 'ALSFRS-R':
['>=36' '[24,36[' '[12,24[' '<12']


In [16]:
def create_sequence_db_df(df):
    records = []
    item_mapping = {}
    item_counter = 1

    for ref, group in df.groupby('REF'):
        parts = []       # for SPMF encoding
        parts_raw = []   # for human‐readable
        for t, (_, row) in enumerate(group.iterrows()):
            # build encoded item list
            items = []
            for col in df.columns:
                if col == 'REF':
                    continue
                value = str(row[col])
                key = f"{col}={value}"
                if key not in item_mapping:
                    item_mapping[key] = item_counter
                    item_counter += 1
                items.append(str(item_mapping[key]))
            parts.append(f"<{t}> " + " ".join(items) + " -1")

            # build raw tuple
            raw_items = " ".join(f"{col} = {row[col]}"
                                 for col in df.columns if col != 'REF')
            parts_raw.append(f"({t}, {raw_items} )")

        sequence_str = " ".join(parts) + " -2"
        sequence_raw = " ".join(parts_raw)
        records.append({
            'REF': ref,
            'Sequence': sequence_str,
            'Sequence_raw': sequence_raw
        })

    # build mapping header
    mapping_str = "@CONVERTED_FROM_TEXT\n"
    for key, number in item_mapping.items():
        mapping_str += f"@ITEM={number}={key}\n"

    return pd.DataFrame(records), mapping_str

In [17]:
sequences, map = create_sequence_db_df(temporal_df)

In [18]:
sequences.head()

,REF,Sequence,Sequence_raw
0,2,<0> 1 2 3 4 5 6 -1 <1> 1 2 3 4 5 6 -1 <2> 1 2 ...,"(0, ALSFRSb = 12 ALSFRSsUL = [8,12[ ALSFRSsT =..."
1,8,<0> 1 8 3 9 5 6 -1 <1> 1 10 11 9 5 6 -1 -2,"(0, ALSFRSb = 12 ALSFRSsUL = [4,8[ ALSFRSsT = ..."
2,9,<0> 1 8 3 7 5 6 -1 <1> 1 10 3 4 5 6 -1 <2> 1 8...,"(0, ALSFRSb = 12 ALSFRSsUL = [4,8[ ALSFRSsT = ..."
3,10,<0> 14 2 15 9 5 6 -1 <1> 14 2 3 9 5 6 -1 <2> 1...,"(0, ALSFRSb = [8,12[ ALSFRSsUL = [8,12[ ALSFRS..."
4,14,<0> 14 8 3 7 12 13 -1 <1> 14 10 11 7 12 13 -1 -2,"(0, ALSFRSb = [8,12[ ALSFRSsUL = [4,8[ ALSFRSs..."


In [19]:
def save_sequences_and_map(sequences_df, mapping_str, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(mapping_str)
        for seq in sequences_df['Sequence']:
            f.write(seq.strip() + '\n')

In [20]:
save_sequences_and_map(sequences, map, 'sequences.txt')

In [56]:
# min_support, min_gap of the time intervals (saltinhos entre os time points), max_gap, min_length of the sequences (quantidade de time points), max_length
os.system("java -jar spmf.jar run Fournier08-Closed+time sequences.txt output.txt 30% 1 1 3 30")

0

In [57]:
import re

def is_subsequence(pat, seq):
    i = 0
    for pset in pat:
        while i < len(seq) and not pset.issubset(seq[i]):
            i += 1
        if i == len(seq): return False
        i += 1
    return True

def parse_sequence(raw):
    out = []
    # find every "(t, …)" chunk
    for content in re.findall(r'\(\s*\d+,\s*(.*?)\s*\)', raw):
        toks = content.split()
        s = set()
        # every three tokens are feat, '=', val
        for i in range(0, len(toks), 3):
            feat, _, val = toks[i : i+3]
            s.add(f"{feat}={val}")
        out.append(s)
    return out

# then your matching block stays the same:
pattern_records = []
with open('output.txt','r', encoding='utf-8') as f:
    for line in f:
        line = line.strip()
        if not line.startswith('<'): continue

        pat_str, sup_str = line.split('#SUP:')
        enc_pattern = pat_str.strip()
        support     = int(sup_str.strip())

        # build decoded pattern
        sets = re.findall(r'<(\d+)>\s+(.*?)\s+-1', enc_pattern)
        decoded = " ".join(
            f"({t}, {' '.join(f'{f} = {v}' for f,v in (x.split('=',1),))} )"
            for t,items in sets
            for x in [" ".join(items.split())]  # no-op to line up formats
        )
        # parse the encoded sets for subsequence
        pat_sets = [ set(s.split()) for _,s in sets ]

        hits = [
            seq_row.REF
            for _, seq_row in sequences.iterrows()
            if is_subsequence(pat_sets, parse_sequence(seq_row.Sequence_raw))
        ]

        pattern_records.append({
            'pattern':         enc_pattern,
            'decoded_pattern': decoded,
            'support':         support,
            'REFs':            hits
        })

df_matches = pd.DataFrame(pattern_records)

df_matches = df_matches.sort_values(            # sort by support desc
    by='support',
    ascending=False
).reset_index(drop=True)

total_sequences = len(sequences)

df_matches['support_pct'] = (df_matches['support'] / total_sequences * 100).round(2)

df_matches.head(50)

,pattern,decoded_pattern,support,REFs,support_pct
0,<0> ALSFRS-R=>=36 -1 <1> ALSFRS-R=>=36 -1 <2> ...,"(0, ALSFRS-R = >=36 ) (1, ALSFRS-R = >=36 ) (2...",249,"[2, 24, 34, 35, 40, 42, 49, 50, 60, 63, 72, 74...",36.03
1,<0> ALSFRS-R=>=36 -1 <1> ALSFRS-R=>=36 -1 <2> ...,"(0, ALSFRS-R = >=36 ) (1, ALSFRS-R = >=36 ) (2...",249,"[2, 24, 34, 35, 40, 42, 45, 49, 50, 64, 66, 72...",36.03
2,<0> R=12 -1 <1> R=12 -1 <2> R=12 -1 <3> R=12 -1,"(0, R = 12 ) (1, R = 12 ) (2, R = 12 ) (3, R =...",248,"[2, 18, 24, 34, 35, 42, 43, 45, 46, 49, 50, 55...",35.89
3,<0> ALSFRS-R=>=36 -1 <1> ALSFRS-R=>=36 -1 <2> ...,"(0, ALSFRS-R = >=36 ) (1, ALSFRS-R = >=36 ) (2...",248,"[2, 24, 34, 35, 40, 42, 45, 46, 49, 50, 64, 66...",35.89
4,<0> ALSFRS-R=>=36 -1 <1> R=12 -1 <2> R=12 -1 <...,"(0, ALSFRS-R = >=36 ) (1, R = 12 ) (2, R = 12 ...",245,"[2, 24, 34, 35, 42, 45, 46, 49, 50, 64, 66, 72...",35.46
5,<0> R=12 -1 <1> ALSFRS-R=>=36 -1 <2> R=12 -1 <...,"(0, R = 12 ) (1, ALSFRS-R = >=36 ) (2, R = 12 ...",242,"[2, 24, 34, 35, 42, 45, 46, 49, 50, 64, 66, 72...",35.02
6,<0> R=12 ALSFRS-R=>=36 -1 <1> ALSFRS-R=>=36 -1...,"(0, R = 12 ALSFRS-R=>=36 ) (1, ALSFRS-R = >=36...",241,"[2, 24, 34, 35, 42, 45, 46, 49, 50, 64, 66, 72...",34.88
7,<0> R=12 -1 <1> ALSFRS-R=>=36 -1 <2> ALSFRS-R=...,"(0, R = 12 ) (1, ALSFRS-R = >=36 ) (2, ALSFRS-...",240,"[2, 24, 34, 35, 42, 49, 50, 60, 63, 72, 74, 78...",34.73
8,<0> R=12 ALSFRS-R=>=36 -1 <1> ALSFRS-R=>=36 -1...,"(0, R = 12 ALSFRS-R=>=36 ) (1, ALSFRS-R = >=36...",239,"[2, 24, 34, 35, 42, 49, 50, 60, 63, 72, 74, 78...",34.59
9,<0> ALSFRS-R=>=36 -1 <1> R=12 -1 <2> ALSFRS-R=...,"(0, ALSFRS-R = >=36 ) (1, R = 12 ) (2, ALSFRS-...",238,"[2, 24, 34, 35, 42, 45, 49, 50, 64, 66, 72, 74...",34.44


### 1.2. Compute the Sequence Database

Note that items have now the form `Feature=value` and you should have a sequence database with as many sequences as patients. 

Each sequence encodes the several time-points (maximum 5) of each patient.

See `Fourier08` example (https://www.philippe-fournier-viger.com/spmf/ClosedSequentialPatterns_TimeConstraints.php) to undertand the format received by the algorithm, specially the time information. 

Remember also the end of the example with PrefixSpan (https://www.philippe-fournier-viger.com/spmf/PrefixSpan.php) to understand how to use strings instead of integers to encode items .

In [12]:
# Write code in cells like this
# ....

Write text in cells like this ...


### 1.3. Compute Sequential Patterns

Use `Fourier08` to compute the closed sequential patterns. Trivial patterns of length 1 should be discarded.

Note that later you need to know what are the sequences (patients) where the patterns occur (the algorithm can output that info).

In [16]:
# Write code in cells like this
# ....

Write text in cells like this ...


## 2.  Learn a classifier to predict NIV using temporal patterns as features together with the static features

In this task you should create a training set where the features are 1) the original static features `(Dataset_static_features.cvs`and 2) the sequential patterns computed above. The class labels to be used for each patient are in file `Dataset_NIV_Evolution_180.cvs`.

### 2.1. Load/Preprocess the Dataset

- Remember to delete from `Dataset_static_features.csv` the patients you deleted in step 1.1. You should only have one row per patient, thus remove repetitions.
- Remember to delete from `Dataset_NIV_Evolution_180.cvs` the patients you deleted in step 1.1.
- Note that for each patient the class label you need from `Dataset_NIV_Evolution_180.cvs` is the one corresponding to the last time-point you considered in step 1.1.

In [20]:
# Write code in cells like this
# ....

Write text in cells like this ...


### 2.2. Create the Training Set

See Section **3.4 Training Set Creation and Model Learning** of the paper Martins et al (2021). 

Note that in this project the original static features are used thus you only need to compute the distance matrix for the sequencial patterns (which are now features), the static features are used as they are.

**Perform the experiments only for binary matrices**.

In [24]:
# Write code in cells like this
# ....

Write text in cells like this ...


#### 2.1.3. Learn the model

See section **4.1 Model Evaluation** of the paper Martins et al (2021).

Use only a `Random-Forest`, default parameters and present the results for **5-fold cross-validation** (mean+-std). 

Note that the problem is difficult so don´t expect high performance.

In [27]:
# Write code in cells like this
# ....

Write text in cells like this 